# Embedding(2) Feature set
former got a lot of cruddle


1. load the test set
2. load the larger corpus (for embedding training)
    - 33k texts
    - (Future) : incorporate additional texts to the embedding training
3. Hyperparameter tuning (OPTUNA package)
    1. Generate the training splits (manual K-foldCV)
    2. Pure embedding (global|average|flatten into single neuron)
    3. Embedding + ANN
    4. Embedding + CNN layer(s)
    5. embedding + RNN (very unstable)
4. Model evaluation(s)
    1. Pure embedding
    2. Embedding + shallow CNN


# 0.5 import and functions

In [73]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# nktk
import nltk

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, matthews_corrcoef, roc_auc_score, f1_score, make_scorer
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle


# tensorflow/keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding, Flatten, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, LSTM
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
import tensorflow as tf
from keras import backend as K

# plotting AUC curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

# embedding
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

# optuna
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
import optuna.visualization as ov

# scikeras
from scikeras.wrappers import KerasClassifier

# stats
from scipy.stats import sem

# saving
import joblib

In [4]:
def average_loss(cv_results):

    loss_list = []; val_loss_list = []

    for i in cv_results['estimator']:

        loss_list.append(i.history_['loss'])
        val_loss_list.append(i.history_['val_loss'])

    loss_array = np.array(loss_list); val_loss_array = np.array(val_loss_list)
    average_loss = loss_array.mean(axis=0); average_val_loss = val_loss_array.mean(axis=0)

    plt.plot(average_loss); plt.plot(average_val_loss)
    plt.ylim([0.0, 5])
    plt.show()

    
    

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    return tokens

def text_to_vector(text_list, word2idx, w2v_model):
    OOV_token = 'OOV'
    
    # replace not found tokens with OOV
    
    new_text = [word if word in word2idx else OOV_token for word in text_list]
    
    return np.array([w2v_model.wv.key_to_index[word] for word in new_text])

def mcc_metric(y_true, y_pred):
    predicted = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    true_pos = tf.math.count_nonzero(predicted * y_true)
    true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
    false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
    false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
    x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
      * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
    mcc = tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)

    if mcc != np.nan:
        return mcc
    else:
        return 0.0

def mcc_metric2(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0.0, 1.5])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
def plot_auc_curve(x_train, x_test, y_train, y_test, model):

    y_test_pred = model.predict(x_test)
    y_train_pred = model.predict(x_train)

    fpr_test, tpr_test, thresholds_keras = roc_curve(y_test, y_test_pred)
    fpr_train, tpr_train, thresholds_keras = roc_curve(y_train, y_train_pred)

    auc_test = auc(fpr_test, tpr_test)
    auc_train = auc(fpr_train, tpr_train)

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr_test, tpr_test, label='train (area = {:.3f})'.format(auc_test))
    plt.plot(fpr_train, tpr_train, label='test (area = {:.3f})'.format(auc_train))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    #plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    
def ann_metrics(model, features, labels):
    
    y_pred = model.predict(features.astype('float64'), batch_size=1, verbose=0)
    #print(y_pred)
    
    y_pred_bool_manual = []
    
    for i in y_pred:
        if i>0.5:
            y_pred_bool_manual.append(1)
        else:
            y_pred_bool_manual.append(0)
    #print(y_pred_bool_manual)
    target_names = ['Irrelevant', 'Relevant']
    res = classification_report(labels, y_pred_bool_manual, target_names=target_names, output_dict=True)
    
    print(classification_report(labels, y_pred_bool_manual, target_names=target_names, zero_division=1))
    
    acc = res['accuracy']
    recall = res['Relevant']['recall']
    precision = res['Relevant']['precision']
    f1 = res['Relevant']['f1-score']
    result = [acc, recall, precision, f1]    
    
    print(result)
    return result

def generate_metrics(history, model_emb, X_train, y_train, X_test, y_test):
    print('train', model_emb.evaluate(X_train, y_train))
    print('train')
    _ = ann_metrics(model_emb, X_train, y_train)


    print('test', model_emb.evaluate(X_test, y_test))
    print('test')
    _ = ann_metrics(model_emb, X_test, y_test)
    
    
    
    
    plot_auc_curve(X_train, X_test, y_train, y_test, model_emb)
    
    #plt.subplot()
    #plt.subplot(ax2)
    plot_loss(history)
    
    
def scale_mcc(mcc):
    return ((mcc-(-1))/(1-(-1)))*(1-0)+0


def custom_f1_pos(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def custom_f1_neg(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tn / (tn + fn + K.epsilon())
    r = tn / (tn + fp + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 2*((p*r)/(p+r+K.epsilon()))


def calculate_f1_score(y, pred, threshold, pos_label):
    # if this also wrong, I compute myself
    y_pred = pred.copy()

    y_pred[y_pred > threshold] = 1
    y_pred[y_pred < threshold] = 0

    return f1_score(y, y_pred, pos_label=pos_label)

# 1. Load test set

In [3]:
test_set = pd.read_json('data/new_test_set_II.json')
val_set = pd.read_json('data/validation_set.json')

test_set.drop(['DOI'], axis=1, inplace=True)
test_set['abstract_tokenized'] = test_set['Abstract'].apply(tokenize)

val_set.drop(['doi'], axis=1, inplace=True)
val_set['abstract_tokenized'] = val_set['abstract'].apply(tokenize)

test_set = test_set.rename(columns={'Abstract': 'abstract', 'Label':'label'})

In [5]:
training1 = pd.read_json('data/new_test_set_II.json')
training2 = pd.read_json('data/validation_set.json')
training1 = training1.rename(columns={'DOI':'doi', 'Abstract': 'abstract', 'Label':'label'})

train = pd.concat([training1, training2]).reset_index(drop=True)

train['abstract_tokenized'] = train['abstract'].apply(tokenize)

# 2. Generate the larger corpus for embedding layer

In [6]:
df = pd.read_json('data/abstract_df_n42188.json')

df = df.dropna()
df.drop(['Author', 'Title', 'DOI', 'scp_id'], axis=1, inplace=True)
df = df.reset_index(drop=True)
df = df.drop_duplicates(subset=['Abstract'], keep='last')

df['Abstract_Tokenized'] = df['Abstract'].apply(tokenize)
larger_corpus = [tokens for tokens in df['Abstract_Tokenized']]

# 3. Hyperparameter search

### 3.0.1 Generate the training splits

In [7]:
five_fold = []

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=1)

for train_index, test_index in kf.split(train):
    t = train.loc[train_index, :]
    v = train.loc[test_index, :]

    five_fold.append((t,v))

## 3.1. Pure embedding

In [36]:
def initiate_embedding(vector_size, window, min_count, epochs, larger_corpus=larger_corpus):
    # Train a Word2Vec model on the larger corpus
    w2v_model = Word2Vec(larger_corpus, vector_size=vector_size,
                                        window=window, 
                                        min_count=min_count, 
                                        workers=4, 
                                        epochs=epochs)
    
    # out of vocabulary token
    OOV_token = 'OOV'

    zero_vector = np.zeros(vector_size)
    index = len(w2v_model.wv.key_to_index)
    w2v_model.wv.key_to_index[OOV_token] = index
    index_to_key = list(w2v_model.wv.index_to_key) + [OOV_token]
    w2v_model.wv.index_to_key = index_to_key
    w2v_model.wv.vectors = np.concatenate((w2v_model.wv.vectors, [zero_vector]), axis=0)

    # Extract the embedding weights from the trained model
    embedding_weights = w2v_model.wv.vectors
    
    return w2v_model, embedding_weights

def features(training_data, model, dim, padding):
    # Step 1: Create a mapping between words in the test set and the corresponding word embeddings
    word2idx = {word: i for i, word in enumerate(model.wv.index_to_key)}

    training_data['keys'] = training_data['abstract_tokenized'].apply(text_to_vector, word2idx=word2idx, w2v_model=model)
    padded = pad_sequences(training_data['keys'], dim, padding=padding, truncating=padding)
    
    X = padded
    y = training_data['label'].values

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)
    
    return X, y
    
def create_embedding_model(trial, dim, embedding_weights):

    # model parameters
    
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)

    dropout = trial.suggest_float('dropout', 0.1, 0.7)
    threshold = trial.suggest_float('threshold', 0.2, 0.8)
    #k_reg = trial.suggest_float('regularizers', 1e-4, 1e-0, log=True)
    
    model_emb = keras.Sequential([keras.layers.Embedding(input_dim=embedding_weights.shape[0],
                                                     output_dim=embedding_weights.shape[1],
                                                     input_length = dim,
                                                     weights=[embedding_weights], trainable=False),
                                

                                  keras.layers.GlobalAveragePooling1D(),
                                  keras.layers.Dropout(dropout),

                                  
                                  #keras.layers.Dense(10, activation='relu', kernel_regularizer=regularizers.L1(k_reg)),

                                  keras.layers.Dense(1, activation='sigmoid')
                                ])
        
    
    model_emb.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy', 
                      metrics=['AUC', keras.metrics.BinaryAccuracy(threshold=threshold),
                                      keras.metrics.Recall(thresholds=threshold),
                                      keras.metrics.Precision(thresholds=threshold),])
    return model_emb


def objective(trial):
    
    # embedding model
    vector_size = trial.suggest_int('vector_size', 55, 100)
    window = trial.suggest_int('context_window', 4, 9)
    min_count = trial.suggest_int('min_count', 4, 6)
    epochs = trial.suggest_int('epochs', 6, 14)
    
    sample_dim = trial.suggest_int('dim', 210, 260)
    padding = trial.suggest_categorical('padding', ['pre', 'post'])
    
    #batch_size = trial.suggest_int('batch_size', 1, 200)
    loss_list = []; auc_list = []; acc_list = []; rec_list = []; prec_list = []; f1_list = []
    
    for train, val in five_fold:
        #print(train, val)

        model, embedding_weights = initiate_embedding(vector_size, window, min_count, epochs)
        
        
        #X_train, y_train, X_test, y_test = features(train, model, sample_dim, padding)
        X_train, y_train = features(train, model, sample_dim, padding)
        X_test, y_test = features(val, model, sample_dim, padding)

        emb_model = create_embedding_model(trial, sample_dim, embedding_weights)

        emb_model.fit(X_train, y_train, verbose=0, validation_split=0.25, epochs=100, batch_size=100)
    
        score = emb_model.evaluate(X_test, y_test, verbose=0)
        
        loss_list.append(score[0])
        auc_list.append(score[1])
        acc_list.append(score[2])
        rec_list.append(score[3])
        prec_list.append(score[4])

        y_pred = emb_model.predict(X_test, verbose=0)
        f1_list.append(calculate_f1_score(y_test, y_pred, threshold=.5, pos_label=1))

    return  np.array(acc_list).mean(), np.array(rec_list).mean(), np.array(f1_list).mean(), np.array(prec_list).mean()


study_pure = optuna.create_study(study_name='pure_embedding',
                                 directions=['maximize', 'maximize', 'maximize', 'maximize'],  
                                 sampler=optuna.samplers.NSGAIISampler())

study_pure.optimize(objective, n_trials=100, n_jobs=1)



[I 2023-09-05 14:17:36,063] A new study created in memory with name: pure_embedding


[I 2023-09-05 14:21:11,115] Trial 0 finished with values: [0.8399999976158142, 0.8968597531318665, 0.8479481177924431, 0.8024441003799438] and parameters: {'vector_size': 97, 'context_window': 7, 'min_count': 4, 'epochs': 7, 'dim': 219, 'padding': 'pre', 'learning_rate': 0.06175283591050675, 'dropout': 0.2641612479032547, 'threshold': 0.39668687769047273}. 
[I 2023-09-05 14:26:43,875] Trial 1 finished with values: [0.8600000023841858, 0.778873085975647, 0.863717519972773, 0.9219106674194336] and parameters: {'vector_size': 85, 'context_window': 5, 'min_count': 6, 'epochs': 13, 'dim': 216, 'padding': 'pre', 'learning_rate': 0.019718734867748347, 'dropout': 0.27571155440689404, 'threshold': 0.6912342726842519}. 
[I 2023-09-05 14:32:37,574] Trial 2 finished with values: [0.5719999969005585, 0.13014911264181137, 0.7986735062108125, 0.95] and parameters: {'vector_size': 63, 'context_window': 8, 'min_count': 4, 'epochs': 13, 'dim': 236, 'padding': 'post', 'learning_rate': 0.00058361126112278

In [130]:
# results
for i in study_pure.best_trials:
    print('model#: ', i.number)
    print('performance: ', i.values)
    print('parameter_set: ', i.params)

model#:  1
performance:  [0.8600000023841858, 0.778873085975647, 0.863717519972773, 0.9219106674194336]
parameter_set:  {'vector_size': 85, 'context_window': 5, 'min_count': 6, 'epochs': 13, 'dim': 216, 'padding': 'pre', 'learning_rate': 0.019718734867748347, 'dropout': 0.27571155440689404, 'threshold': 0.6912342726842519}
model#:  4
performance:  [0.5119999885559082, 0.9911564588546753, 0.7843624100898563, 0.5008643209934235]
parameter_set:  {'vector_size': 72, 'context_window': 5, 'min_count': 4, 'epochs': 11, 'dim': 247, 'padding': 'post', 'learning_rate': 0.00041035703969044996, 'dropout': 0.6064450125924121, 'threshold': 0.3508595938250224}
model#:  12
performance:  [0.7839999914169311, 0.9663189649581909, 0.8620703359017445, 0.7018375992774963]
parameter_set:  {'vector_size': 83, 'context_window': 8, 'min_count': 5, 'epochs': 10, 'dim': 231, 'padding': 'pre', 'learning_rate': 0.024914324150988945, 'dropout': 0.3266837646724929, 'threshold': 0.23028018057396393}
model#:  15
perfor

In [ ]:
pure_df = study_pure.trials_dataframe()
pure_df = pure_df.rename(columns=({'values_0': 'accuracy', 'values_1': 'recall', 'values_2': 'f1', 'values_3': 'precison'}))
pure_df.to_json('data/hyperparameter_search/pure_embedding.json')

joblib.dump(study_pure, 'data/hyperparameter_search/pure_embedding.pkl')

In [ ]:
pure_from_disk = joblib.load('data/hyperparameter_search/pure_embedding.pkl')

In [9]:
# results
for i in study_pure.best_trials:
    print('model#: ', i.number)
    print('performance: ', i.values)
    print('parameter_set: ', i.params)

model#:  1
performance:  [0.37018485069274903, 0.8480000019073486, 0.9172630667686462, 0.884265398979187]
parameter_set:  {'vector_size': 95, 'context_window': 9, 'min_count': 4, 'epochs': 8, 'dim': 224, 'padding': 'pre', 'learning_rate': 0.013121832123684417, 'dropout': 0.4402980193837679, 'threshold': 0.5099871650242483}
model#:  4
performance:  [0.4063465356826782, 0.8580000042915344, 0.913949716091156, 0.8936826527118683]
parameter_set:  {'vector_size': 98, 'context_window': 8, 'min_count': 4, 'epochs': 7, 'dim': 217, 'padding': 'pre', 'learning_rate': 0.002104272243300588, 'dropout': 0.5665904244287674, 'threshold': 0.4837522534769876}
model#:  16
performance:  [0.37377700209617615, 0.6839999914169311, 0.9236754536628723, 0.8780072510242463]
parameter_set:  {'vector_size': 67, 'context_window': 6, 'min_count': 4, 'epochs': 10, 'dim': 253, 'padding': 'pre', 'learning_rate': 0.006558954145842335, 'dropout': 0.3398679639334519, 'threshold': 0.1592588728061565}
model#:  18
performance

## 3.2. Embedding + CNN

In [219]:
def initiate_embedding(vector_size, window, min_count, epochs, larger_corpus=larger_corpus):
    # Train a Word2Vec model on the larger corpus
    w2v_model = Word2Vec(larger_corpus, vector_size=vector_size, window=window, min_count=min_count, workers=4, epochs=epochs)
    
    # out of vocabulary token
    OOV_token = 'OOV'

    zero_vector = np.zeros(vector_size)
    index = len(w2v_model.wv.key_to_index)
    w2v_model.wv.key_to_index[OOV_token] = index
    index_to_key = list(w2v_model.wv.index_to_key) + [OOV_token]
    w2v_model.wv.index_to_key = index_to_key
    w2v_model.wv.vectors = np.concatenate((w2v_model.wv.vectors, [zero_vector]), axis=0)

    # Extract the embedding weights from the trained model
    embedding_weights = w2v_model.wv.vectors
    
    return w2v_model, embedding_weights

def features(training_data, model, dim, padding):
    # Step 1: Create a mapping between words in the test set and the corresponding word embeddings
    word2idx = {word: i for i, word in enumerate(model.wv.index_to_key)}

    training_data['keys'] = training_data['abstract_tokenized'].apply(text_to_vector, word2idx=word2idx, w2v_model=model)
    padded = pad_sequences(training_data['keys'], dim, padding=padding, truncating=padding)
    
    X = padded
    y = training_data['label'].values

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    return X, y
    
def create_embedding_model(trial, dim, embedding_weights):

    # model parameters
    
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)

    dropout = trial.suggest_float('dropout', 0.1, 0.7)
    #threshold = trial.suggest_float('threshold', 0.1, 0.8)
    threshold=0.5

    filters = trial.suggest_int('filter', 1, 64)
    kernel_size = trial.suggest_int('kernel', 1, 20)
    #final_dense = trial.suggest_int('units', 10,30)

    k_reg = trial.suggest_float('regularizer', 1e-5, 1e-0, log=True)
    
    #embedding_weights = initiate_embedding()

    
    model_emb = keras.Sequential([keras.layers.Embedding(input_dim=embedding_weights.shape[0],
                                                     output_dim=embedding_weights.shape[1],
                                                     input_length = dim,
                                                     weights=[embedding_weights], trainable=False),
                                  keras.layers.Conv1D(filters=filters,
                                                      kernel_size=kernel_size, 
                                                      activation='relu',
                                                      kernel_regularizer=regularizers.L1(k_reg)),

                                  keras.layers.MaxPooling1D(pool_size=2),
                                  keras.layers.Flatten(),
                                  #keras.layers.Dense(final_dense, activation='relu'),

                                  #keras.layers.GlobalAveragePooling1D(),
                                  keras.layers.Dropout(dropout),

                                  keras.layers.Dense(1, activation='sigmoid')
                                ])
        
    
    model_emb.compile(optimizer = keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy', 
                      metrics=['AUC', keras.metrics.BinaryAccuracy(threshold=threshold),
                                 keras.metrics.Recall(thresholds=threshold),
                                 keras.metrics.Precision(thresholds=threshold)])
    
    return model_emb
    
    
def objective(trial):
    
    # embedding model
    vector_size = trial.suggest_int('vector_size', 10, 100)
    window = trial.suggest_int('context_window', 2, 12)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 6, 14)
    
    sample_dim = trial.suggest_int('dim', 35, 440)
    padding = trial.suggest_categorical('padding', ['pre', 'post'])

    loss_list = []; auc_list = []; rec_list = []; acc_list = []; prec_list=[]; f1_list=[]
    
    for train, val in five_fold:

        model, embedding_weights = initiate_embedding(vector_size, window, min_count, epochs)
        
        X_train, y_train = features(train, model, sample_dim, padding)
        X_test, y_test = features(val, model, sample_dim, padding)
        
        emb_model = create_embedding_model(trial, sample_dim, embedding_weights)

        emb_model.fit(X_train, y_train, validation_split=0.25, verbose=0, epochs=100, batch_size=100)
    
        score = emb_model.evaluate(X_test, y_test, verbose=0)
        
        loss_list.append(score[0])
        auc_list.append(score[1])
        
        acc_list.append(score[2])
        rec_list.append(score[3])
        prec_list.append(score[4])


        y_pred = emb_model.predict(X_test, verbose=0)

        f1_list.append(calculate_f1_score(y_test, y_pred, threshold=.5, pos_label=1))

    return  np.array(acc_list).mean(), np.array(rec_list).mean(), np.array(f1_list).mean(), np.array(prec_list).mean()

    #return np.array(loss_list).mean(), np.array(acc_list).mean(), np.array(auc_list).mean(), np.array(mcc_list).mean()


study_cnn = optuna.create_study(study_name='embedding_cnn', 
                            directions=['maximize', 'maximize', 'maximize', 'maximize'], 
                            sampler=optuna.samplers.NSGAIISampler())

study_cnn.optimize(objective, n_trials=100, n_jobs=1)

[I 2023-09-08 10:13:42,804] A new study created in memory with name: embedding_cnn


[I 2023-09-08 10:27:06,115] Trial 0 finished with values: [0.7840000033378601, 0.7426276206970215, 0.7697428192257428, 0.8223498225212097] and parameters: {'vector_size': 34, 'context_window': 9, 'min_count': 4, 'epochs': 6, 'dim': 358, 'padding': 'pre', 'learning_rate': 0.010714528771518971, 'dropout': 0.6535637196987211, 'filter': 30, 'kernel': 12, 'regularizer': 0.015112953378004324}. 
[I 2023-09-08 10:40:11,720] Trial 1 finished with values: [0.8560000061988831, 0.8530639767646789, 0.8523795728960095, 0.8540852665901184] and parameters: {'vector_size': 90, 'context_window': 10, 'min_count': 3, 'epochs': 6, 'dim': 225, 'padding': 'pre', 'learning_rate': 0.000944635755123106, 'dropout': 0.36577222193378545, 'filter': 13, 'kernel': 8, 'regularizer': 2.3799437817752373e-05}. 
[I 2023-09-08 10:52:01,588] Trial 2 finished with values: [0.8020000219345093, 0.795356285572052, 0.7733935361223606, 0.8121090173721314] and parameters: {'vector_size': 27, 'context_window': 10, 'min_count': 2, '

In [212]:
# results over 500 tries
for i in study_cnn.best_trials:
    print('model#: ', i.number)
    print('performance: ', i.values)
    print('parameter_set: ', i.params)

model#:  39
performance:  [0.8240000009536743, 0.8557191014289856, 0.8245682557206406, 0.7993919610977173]
parameter_set:  {'vector_size': 54, 'context_window': 9, 'min_count': 9, 'epochs': 12, 'dim': 220, 'padding': 'pre', 'learning_rate': 0.005025068172771905, 'dropout': 0.17097448392525408, 'filter': 53, 'kernel': 12}
model#:  41
performance:  [0.8379999876022339, 0.8529966354370118, 0.836459109949163, 0.8237209916114807]
parameter_set:  {'vector_size': 36, 'context_window': 9, 'min_count': 6, 'epochs': 14, 'dim': 296, 'padding': 'pre', 'learning_rate': 0.002957173541305207, 'dropout': 0.6813952823962288, 'filter': 33, 'kernel': 19}
model#:  43
performance:  [0.8359999895095825, 0.8327533960342407, 0.8321374828821467, 0.8326308608055115]
parameter_set:  {'vector_size': 68, 'context_window': 3, 'min_count': 3, 'epochs': 9, 'dim': 437, 'padding': 'pre', 'learning_rate': 0.000149405671204404, 'dropout': 0.41186943857120706, 'filter': 44, 'kernel': 17}
model#:  76
performance:  [0.84000

In [229]:
cnn_df = study_cnn.trials_dataframe()
cnn_df = pure_df.rename(columns=({'values_0': 'accuracy', 'values_1': 'recall', 'values_2': 'f1', 'values_3': 'precison'}))
cnn_df.to_json('data/hyperparameter_search/embedding_conv1D/embedding_conv1D.json')

joblib.dump(study_cnn, 'data/hyperparameter_search/embedding_conv1D/embedding_conv1D.pkl')

['data/hyperparameter_search/embedding_conv1D/embedding_conv1D.pkl']

# 4. Model Evaluation

### 4.0.1 Generate the training/test splits

In [169]:
# import the unseen test for final testing of the models
test = pd.read_json('data/unseen_test.json')

if 'doi' in test.columns:
    test = test.drop(columns='doi')

test['abstract_tokenized'] = test['abstract'].apply(tokenize)

## 4.1. Pure embedding evaluation

In [158]:
def initiate_embedding(vector_size, window, min_count, epochs, larger_corpus=larger_corpus):
    # Train a Word2Vec model on the larger corpus
    w2v_model = Word2Vec(larger_corpus, vector_size=vector_size,
                         window=window, min_count=min_count,
                         workers=4, epochs=epochs)
    
    # out of vocabulary token
    OOV_token = 'OOV'

    zero_vector = np.zeros(vector_size)
    index = len(w2v_model.wv.key_to_index)
    w2v_model.wv.key_to_index[OOV_token] = index
    index_to_key = list(w2v_model.wv.index_to_key) + [OOV_token]
    w2v_model.wv.index_to_key = index_to_key
    w2v_model.wv.vectors = np.concatenate((w2v_model.wv.vectors, [zero_vector]), axis=0)

    # Extract the embedding weights from the trained model
    embedding_weights = w2v_model.wv.vectors
    
    return w2v_model, embedding_weights

def generate_features(training_data, model, dim, padding):
    
    word2idx = {word: i for i, word in enumerate(model.wv.index_to_key)}
    
    
    training_data['keys'] = training_data['abstract_tokenized'].apply(text_to_vector, word2idx=word2idx, w2v_model=model)
    
    padded = pad_sequences(training_data['keys'], dim, padding=padding, truncating=padding)
    
    X = padded
    y = training_data['label'].values

    
    
    return X, y

def get_emb_model(embedding_weights, lr, do, threshold, dim):
    
    

    model = keras.Sequential([keras.layers.Embedding(input_dim=embedding_weights.shape[0],
                                                     output_dim=embedding_weights.shape[1],
                                                     input_length = dim,
                                                     weights=[embedding_weights], trainable=False),
                              
                              keras.layers.GlobalAveragePooling1D(),
                              keras.layers.Dropout(do),
                              keras.layers.Dense(1, activation='sigmoid')
                             ])
        
    
    model.compile(optimizer = keras.optimizers.Adam(learning_rate=lr),
                      loss='binary_crossentropy', 
                      metrics=['AUC',
                            keras.metrics.BinaryAccuracy(threshold=threshold),
                            keras.metrics.Recall(thresholds=threshold),
                            keras.metrics.Precision(thresholds=threshold)])



    return model

# parametersets
# 1. {'vector_size': 95, 'context_window': 9, 'min_count': 4, 'epochs': 8, 'dim': 224, 'padding': 'pre', 'learning_rate': 0.013121832123684417, 'dropout': 0.4402980193837679, 'threshold': 0.5099871650242483}
# 2. {'vector_size': 98, 'context_window': 8, 'min_count': 4, 'epochs': 7, 'dim': 217, 'padding': 'pre', 'learning_rate': 0.002104272243300588, 'dropout': 0.5665904244287674, 'threshold': 0.4837522534769876}
# ?. {'vector_size': 61, 'context_window': 5, 'min_count': 5, 'epochs': 12, 'dim': 215, 'padding': 'post', 'learning_rate': 0.05715250774676652, 'dropout': 0.3106013643470691, 'threshold': 0.3206284734965039}

def test_models(vector_size, context_window, min_count, epochs, dim, padding, learning_rate, dropout, threshold):

    #res_dict = {}
    temp_dict = {}
    train_dict = {}; test_dict = {}

    tr_loss_list = []; tr_auc_list = []; tr_prec_list = []; tr_acc_list = []; tr_rec_list = []; tr_fpos_list = []; tr_fneg_list = []
    te_loss_list = []; te_auc_list = []; te_prec_list = []; te_acc_list = []; te_rec_list = []; te_fpos_list = []; te_fneg_list = []
    

    for rs in range(10):

        #train_set, test_set = do_shuffle_dataset(rs, df=df)
        
        #for train, test in zip(train_set, test_set):

        model, weights = initiate_embedding(vector_size=vector_size, window=context_window, min_count=min_count, epochs=epochs)

        X, y = generate_features(train, model=model, dim=dim, padding=padding)
        X_test, y_test = generate_features(test, model=model, dim=dim, padding=padding)

        emb_model = get_emb_model(weights, 
                                lr=learning_rate, 
                                do=dropout, 
                                threshold=threshold, 
                                dim=dim)
        
        emb_model.fit(X, y, validation_split=0.0, verbose=0, epochs=100, batch_size=100)

        tr_score = emb_model.evaluate(X, y, verbose=0)
        te_score = emb_model.evaluate(X_test, y_test, verbose=0)

        ################# Manually handle the f1-score #####################
        # train
        pred = emb_model.predict(X, verbose=0) # y is true
        f1_train_pos = calculate_f1_score(y, pred, threshold=threshold, pos_label=1)
        f1_trian_neg = calculate_f1_score(y, pred, threshold=threshold, pos_label=0)

        # test
        test_pred = emb_model.predict(X_test, verbose=0)
        f1_test_pos = calculate_f1_score(y_test, test_pred, threshold=threshold, pos_label=1)
        f1_test_neg = calculate_f1_score(y_test, test_pred, threshold=threshold, pos_label=0)
        
        # save the f1-scores
        tr_fpos_list.append(f1_train_pos); tr_fneg_list.append(f1_trian_neg)
        te_fpos_list.append(f1_test_pos); te_fneg_list.append(f1_test_neg)

        tr_loss_list.append(tr_score[0]); tr_auc_list.append(tr_score[1]); tr_acc_list.append(tr_score[2]); tr_rec_list.append(tr_score[3]); tr_prec_list.append(tr_score[4]); 
        te_loss_list.append(te_score[0]); te_auc_list.append(te_score[1]); te_acc_list.append(te_score[2]); te_rec_list.append(te_score[3]); te_prec_list.append(te_score[4]); 

        #for i in te_mcc_list:
        #    if np.isnan(i):
        #        print(emb_model.predict(X_test).values, y_test.values)
        
    train_dict['loss'] = tr_loss_list; train_dict['auc'] = tr_auc_list; train_dict['prec'] = tr_prec_list; train_dict['acc'] = tr_acc_list; train_dict['rec'] = tr_rec_list; train_dict['f1pos'] = tr_fpos_list; train_dict['f1neg'] = tr_fneg_list
    test_dict['loss'] = te_loss_list; test_dict['auc'] = te_auc_list; test_dict['prec'] = te_prec_list; test_dict['acc'] = te_acc_list; test_dict['rec'] = te_rec_list; test_dict['f1pos'] = te_fpos_list; test_dict['f1neg'] = te_fneg_list


    temp_dict['train'] = train_dict
    temp_dict['test'] = test_dict

    #res_dict[str(rs)] = temp_dict
        
    return temp_dict


In [181]:
# filtering, to get the best among the stores "best_trials"

for i in study_pure.best_trials:
    #print(i.params, ',')
    if i.values[0] > 0.85:
        if i.values[1] > 0.85:
            if i.values[2] > 0.85:
                if i.values[3] > 0.85:
                    #print(i.values)
                    print(i.params, ',')


{'vector_size': 77, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'post', 'learning_rate': 0.03176099025801702, 'dropout': 0.5136744356192664, 'threshold': 0.5848579651306551} ,
{'vector_size': 56, 'context_window': 6, 'min_count': 6, 'epochs': 14, 'dim': 212, 'padding': 'pre', 'learning_rate': 0.0026738550368240127, 'dropout': 0.36981250642348884, 'threshold': 0.48168409474586804} ,
{'vector_size': 70, 'context_window': 6, 'min_count': 4, 'epochs': 11, 'dim': 248, 'padding': 'pre', 'learning_rate': 0.008076709312390297, 'dropout': 0.45592506038880143, 'threshold': 0.5188836233727432} ,
{'vector_size': 57, 'context_window': 8, 'min_count': 4, 'epochs': 8, 'dim': 248, 'padding': 'post', 'learning_rate': 0.0927796322645081, 'dropout': 0.3035608419348366, 'threshold': 0.598413172203673} ,
{'vector_size': 89, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'pre', 'learning_rate': 0.008200999065076656, 'dropout': 0.2368719869678908, 'thres

In [182]:
hyper_list = [{'vector_size': 77, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'post', 'learning_rate': 0.03176099025801702, 'dropout': 0.5136744356192664, 'threshold': 0.5848579651306551} ,
{'vector_size': 56, 'context_window': 6, 'min_count': 6, 'epochs': 14, 'dim': 212, 'padding': 'pre', 'learning_rate': 0.0026738550368240127, 'dropout': 0.36981250642348884, 'threshold': 0.48168409474586804} ,
{'vector_size': 70, 'context_window': 6, 'min_count': 4, 'epochs': 11, 'dim': 248, 'padding': 'pre', 'learning_rate': 0.008076709312390297, 'dropout': 0.45592506038880143, 'threshold': 0.5188836233727432} ,
{'vector_size': 57, 'context_window': 8, 'min_count': 4, 'epochs': 8, 'dim': 248, 'padding': 'post', 'learning_rate': 0.0927796322645081, 'dropout': 0.3035608419348366, 'threshold': 0.598413172203673} ,
{'vector_size': 89, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'pre', 'learning_rate': 0.008200999065076656, 'dropout': 0.2368719869678908, 'threshold': 0.49106574275893305}
]

single_res = [{'vector_size': 61, 'context_window': 5, 'min_count': 5, 'epochs': 12, 'dim': 215, 'padding': 'post', 'learning_rate': 0.05715250774676652, 'dropout': 0.3106013643470691, 'threshold': 0.3206284734965039},
{'vector_size': 67, 'context_window': 6, 'min_count': 4, 'epochs': 10, 'dim': 253, 'padding': 'pre', 'learning_rate': 0.006558954145842335, 'dropout': 0.3398679639334519, 'threshold': 0.1592588728061565}
]

hyper_dict2 = {}

for index, h in enumerate(hyper_list):
    print('current_para_set', h)
    hyper_dict2['set_' + str(index)] = test_models(vector_size=h['vector_size'],
                                                  context_window=h['context_window'],
                                                  min_count=h['min_count'],
                                                  epochs=h['epochs'],
                                                  dim=h['dim'],
                                                  padding=h['padding'],
                                                  learning_rate=h['learning_rate'],
                                                  dropout=h['dropout'],
                                                  threshold=h['threshold'])


    #print(hyper_dict2['set_' + str(index)])

# best
# {'vector_size': 89, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'pre', 'learning_rate': 0.008200999065076656, 'dropout': 0.2368719869678908, 'threshold': 0.49106574275893305}

current_para_set {'vector_size': 77, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'post', 'learning_rate': 0.03176099025801702, 'dropout': 0.5136744356192664, 'threshold': 0.5848579651306551}
current_para_set {'vector_size': 56, 'context_window': 6, 'min_count': 6, 'epochs': 14, 'dim': 212, 'padding': 'pre', 'learning_rate': 0.0026738550368240127, 'dropout': 0.36981250642348884, 'threshold': 0.48168409474586804}
current_para_set {'vector_size': 70, 'context_window': 6, 'min_count': 4, 'epochs': 11, 'dim': 248, 'padding': 'pre', 'learning_rate': 0.008076709312390297, 'dropout': 0.45592506038880143, 'threshold': 0.5188836233727432}
current_para_set {'vector_size': 57, 'context_window': 8, 'min_count': 4, 'epochs': 8, 'dim': 248, 'padding': 'post', 'learning_rate': 0.0927796322645081, 'dropout': 0.3035608419348366, 'threshold': 0.598413172203673}
current_para_set {'vector_size': 89, 'context_window': 7, 'min_count': 5, 'epochs': 6, 'dim': 227, 'padding': 'pre',

In [183]:
for key, value in hyper_dict2.items():
    print(key)
    for key2, value2 in value.items():

        for metric, li in value2.items():
            if key2 == 'train':
                print('train', metric, np.array(li).mean(), sem(np.array(li)))
            if key2 == 'test':
                print('test', metric, np.array(li).mean(), sem(np.array(li)))


set_0
train loss 0.3056305289268494 0.002268255131893136
train auc 0.9475037932395936 0.0006035596194860917
train prec 0.9055914998054504 0.00513317209897527
train acc 0.8804000020027161 0.0021249822319193094
train rec 0.8444898009300232 0.006452192446274809
train f1pos 0.8736755081110615 0.002460209711773957
train f1neg 0.8863811053796666 0.0020446684703642093
test loss 0.2626379206776619 0.00390934569660372
test auc 0.9678071558475494 0.0017197223540512197
test prec 0.9340299665927887 0.0029115654135801867
test acc 0.9240000069141387 0.004521552525286969
test rec 0.9156862795352936 0.008293187440621807
test f1pos 0.9246028770297844 0.004729714772020905
test f1neg 0.9233618877193841 0.004311437332183459
set_1
train loss 0.33589772284030917 0.0014247033649381175
train auc 0.9395566046237945 0.000536658546469268
train prec 0.8666624665260315 0.0024020510253922787
train acc 0.8782000064849853 0.0014742248236828857
train rec 0.8881632506847381 0.0022928337757458475
train f1pos 0.877245337

## 4.2. CNN embedding testing

In [226]:
def initiate_embedding(vector_size, window, min_count, epochs, larger_corpus=larger_corpus):
    # Train a Word2Vec model on the larger corpus
    w2v_model = Word2Vec(larger_corpus, vector_size=vector_size,
                         window=window, min_count=min_count,
                         workers=4, epochs=epochs)
    
    # out of vocabulary token
    OOV_token = 'OOV'

    zero_vector = np.zeros(vector_size)
    index = len(w2v_model.wv.key_to_index)
    w2v_model.wv.key_to_index[OOV_token] = index
    index_to_key = list(w2v_model.wv.index_to_key) + [OOV_token]
    w2v_model.wv.index_to_key = index_to_key
    w2v_model.wv.vectors = np.concatenate((w2v_model.wv.vectors, [zero_vector]), axis=0)

    # Extract the embedding weights from the trained model
    embedding_weights = w2v_model.wv.vectors
    
    return w2v_model, embedding_weights

def generate_features(training_data, model, dim, padding):
    
    word2idx = {word: i for i, word in enumerate(model.wv.index_to_key)}
    
    training_data['keys'] = training_data['abstract_tokenized'].apply(text_to_vector, word2idx=word2idx, w2v_model=model)
    
    padded = pad_sequences(training_data['keys'], dim, padding=padding, truncating=padding)
    
    X = padded
    y = training_data['label'].values

    
    
    return X, y

def get_eCNN_model(embedding_weights, lr, do, threshold, dim, filter, kernel_size, units, regularizer):
    
    

    model_eCNN = keras.Sequential([keras.layers.Embedding(input_dim=embedding_weights.shape[0],
                                                     output_dim=embedding_weights.shape[1],
                                                     input_length = dim,
                                                     weights=[embedding_weights], trainable=False),
                                  keras.layers.Conv1D(filters=filter,
                                                      kernel_size=kernel_size, 
                                                      activation='relu',
                                                      kernel_regularizer=regularizers.L1(regularizer)),

                                  keras.layers.MaxPooling1D(pool_size=2),
                                  keras.layers.Flatten(),
                                  #keras.layers.Dense(units, activation='relu'),

                                  #keras.layers.GlobalAveragePooling1D(),
                                  keras.layers.Dropout(do),

                                  keras.layers.Dense(1, activation='sigmoid')
                                ])
        
    
    model_eCNN.compile(optimizer = keras.optimizers.Adam(learning_rate=lr),
                      loss='binary_crossentropy', 
                      metrics=['AUC',
                            keras.metrics.BinaryAccuracy(threshold=threshold),
                            keras.metrics.Recall(thresholds=threshold),
                            keras.metrics.Precision(thresholds=threshold)])

    return model_eCNN

# parametersets
# 1. {'vector_size': 95, 'context_window': 9, 'min_count': 4, 'epochs': 8, 'dim': 224, 'padding': 'pre', 'learning_rate': 0.013121832123684417, 'dropout': 0.4402980193837679, 'threshold': 0.5099871650242483}
# 2. {'vector_size': 98, 'context_window': 8, 'min_count': 4, 'epochs': 7, 'dim': 217, 'padding': 'pre', 'learning_rate': 0.002104272243300588, 'dropout': 0.5665904244287674, 'threshold': 0.4837522534769876}


def test_eCNN_model(vector_size, context_window, min_count, epochs, dim, padding, learning_rate, dropout, threshold, filter, kernel_size, units, regularizer):
    
    res_dict = {}
    temp_dict = {}
    tr_loss_list = []; tr_auc_list = []; tr_prec_list = []; tr_acc_list = []; tr_rec_list = []; tr_fpos_list = []; tr_fneg_list = []
    te_loss_list = []; te_auc_list = []; te_prec_list = []; te_acc_list = []; te_rec_list = []; te_fpos_list = []; te_fneg_list = []
        
    #train_set, test_set = do_shuffle_dataset(rs, df=df)

    train_dict = {}; test_dict = {}


    for rs in range(10):
        
        model, weights = initiate_embedding(vector_size=vector_size, window=context_window, min_count=min_count, epochs=epochs)

        X, y = generate_features(train, model=model, dim=dim, padding=padding)
        X_test, y_test = generate_features(test, model=model, dim=dim, padding=padding)

        eCNN_model = get_eCNN_model(weights, 
                                lr=learning_rate, 
                                do=dropout, 
                                threshold=threshold, 
                                dim=dim,
                                filter=filter, 
                                kernel_size=kernel_size,
                                units=units,
                                regularizer=regularizer)
        
        eCNN_model.fit(X, y, validation_split=0.0, verbose=0, epochs=100, batch_size=100)

        tr_score = eCNN_model.evaluate(X, y, verbose=0)
        te_score = eCNN_model.evaluate(X_test, y_test, verbose=0)

        ################# Manually handle the f1-score #####################
        # train
        pred = eCNN_model.predict(X, verbose=0) # y is true
        f1_train_pos = calculate_f1_score(y, pred, threshold=threshold, pos_label=1)
        f1_trian_neg = calculate_f1_score(y, pred, threshold=threshold, pos_label=0)

        # test
        test_pred = eCNN_model.predict(X_test, verbose=0)
        f1_test_pos = calculate_f1_score(y_test, test_pred, threshold=threshold, pos_label=1)
        f1_test_neg = calculate_f1_score(y_test, test_pred, threshold=threshold, pos_label=0)
        
        # save the f1-scores
        tr_fpos_list.append(f1_train_pos); tr_fneg_list.append(f1_trian_neg)
        te_fpos_list.append(f1_test_pos); te_fneg_list.append(f1_test_neg)
        
        tr_loss_list.append(tr_score[0]); tr_auc_list.append(tr_score[1]); tr_acc_list.append(tr_score[2]); tr_rec_list.append(tr_score[3]); tr_prec_list.append(tr_score[4]); 
        te_loss_list.append(te_score[0]); te_auc_list.append(te_score[1]); te_acc_list.append(te_score[2]); te_rec_list.append(te_score[3]); te_prec_list.append(te_score[4]); 

        #for i in te_mcc_list:
        #    if np.isnan(i):
        #        print(emb_model.predict(X_test).values, y_test.values)
        
    train_dict['loss'] = tr_loss_list; train_dict['auc'] = tr_auc_list; train_dict['prec'] = tr_prec_list; train_dict['acc'] = tr_acc_list; train_dict['rec'] = tr_rec_list; train_dict['f1pos'] = tr_fpos_list; train_dict['f1neg'] = tr_fneg_list
    test_dict['loss'] = te_loss_list; test_dict['auc'] = te_auc_list; test_dict['prec'] = te_prec_list; test_dict['acc'] = te_acc_list; test_dict['rec'] = te_rec_list; test_dict['f1pos'] = te_fpos_list; test_dict['f1neg'] = te_fneg_list


    temp_dict['train'] = train_dict
    temp_dict['test'] = test_dict


    return temp_dict

In [220]:
for i in study_cnn.best_trials:
    print(i.params, ',')

{'vector_size': 24, 'context_window': 6, 'min_count': 6, 'epochs': 12, 'dim': 309, 'padding': 'pre', 'learning_rate': 0.002008567149888518, 'dropout': 0.25961725113431555, 'filter': 64, 'kernel': 3, 'regularizer': 0.49787873612648437} ,
{'vector_size': 57, 'context_window': 10, 'min_count': 7, 'epochs': 6, 'dim': 225, 'padding': 'post', 'learning_rate': 0.0004312947164406039, 'dropout': 0.36577222193378545, 'filter': 13, 'kernel': 10, 'regularizer': 0.08165609142696244} ,
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.49787873612648437} ,


In [249]:
hyper_list = [{'vector_size': 24, 'context_window': 6, 'min_count': 6, 'epochs': 12, 'dim': 309, 'padding': 'pre', 'learning_rate': 0.002008567149888518, 'dropout': 0.25961725113431555, 'filter': 64, 'kernel': 3, 'regularizer': 0.49787873612648437} ,
{'vector_size': 57, 'context_window': 10, 'min_count': 7, 'epochs': 6, 'dim': 225, 'padding': 'post', 'learning_rate': 0.0004312947164406039, 'dropout': 0.36577222193378545, 'filter': 13, 'kernel': 10, 'regularizer': 0.08165609142696244} ,
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.49787873612648437},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.1}]

threshold_adjusting = [{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.2},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.3},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.4},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.5},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.6},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.7},
{'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.8}]

eCNN_dict = {}

for index, h in enumerate(threshold_adjusting):
    print('current_para_set', h)
    #h['threshold'] = 0.3
    h['units'] = 1
    eCNN_dict['set_' + str(index)] = test_eCNN_model(vector_size=h['vector_size'],
                                                  context_window=h['context_window'],
                                                  min_count=h['min_count'],
                                                  epochs=h['epochs'],
                                                  dim=h['dim'],
                                                  padding=h['padding'],
                                                  learning_rate=h['learning_rate'],
                                                  dropout=h['dropout'],
                                                  threshold=h['threshold'],
                                                  filter=h['filter'],
                                                  kernel_size=h['kernel'],
                                                  units=h['units'],
                                                  regularizer=h['regularizer']  
                                                  )


    print(eCNN_dict['set_' + str(index)])


# best performing set
# {'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold':0.5},

# is outperformed by a simpler model

current_para_set {'vector_size': 24, 'context_window': 10, 'min_count': 6, 'epochs': 11, 'dim': 165, 'padding': 'post', 'learning_rate': 0.00553324031182354, 'dropout': 0.25961725113431555, 'filter': 36, 'kernel': 3, 'regularizer': 0.5, 'threshold': 0.2}
{'train': {'loss': [1.2730263471603394, 1.3182662725448608, 1.2680500745773315, 1.386303424835205, 1.2588427066802979, 1.2660224437713623, 1.352420687675476, 1.2406877279281616, 1.3105542659759521, 1.32267165184021], 'auc': [0.9422088861465454, 0.9443057775497437, 0.945554256439209, 0.9468827247619629, 0.9458503723144531, 0.9393036961555481, 0.9490676522254944, 0.9426730871200562, 0.9430732727050781, 0.9426010847091675], 'prec': [0.7267080545425415, 0.6713880896568298, 0.6539509296417236, 0.6014851331710815, 0.7169230580329895, 0.6911764740943909, 0.6666666865348816, 0.6648044586181641, 0.6487935781478882, 0.6363636255264282], 'acc': [0.8019999861717224, 0.7519999742507935, 0.7360000014305115, 0.6740000247955322, 0.7919999957084656, 0.

In [ ]:
Conv1D(
    filters=10,
    kernel_size=10, 
    activation='relu',
    kernel_regularizer=1)

In [257]:
for key, value in eCNN_dict.items():
    print(key)
    if key == 'set_3':
        for key2, value2 in value.items():

            for metric, li in value2.items():
                if key2 == 'train':
                    print('train', metric, np.array(li).mean(), sem(np.array(li)), np.array(li).std())

                if key2 == 'test':
                    print('test', metric, np.array(li).mean(), sem(np.array(li)), np.array(li).std())

set_0
set_1
set_2
set_3
train loss 1.2667137145996095 0.006505596821005861 0.019516790463017585
train auc 0.9474365592002869 0.0006623298162016013 0.001986989448604804
train prec 0.880690723657608 0.010375513639304381 0.031126540917913146
train acc 0.8832000017166137 0.002768073235353604 0.008304219706060812
train rec 0.88408163189888 0.012546551977426358 0.037639655932279074
train f1pos 0.8810952533400837 0.002921339637385103 0.00876401891215531
train f1neg 0.8848853281372324 0.0034821320948246305 0.010446396284473893
test loss 1.2405323147773744 0.0058845249006613904 0.01765357470198417
test auc 0.9658063471317291 0.0014289957151290542 0.004286987145387162
test prec 0.906563937664032 0.00992378721213859 0.02977136163641577
test acc 0.9140000104904175 0.004268751316126819 0.01280625394838046
test rec 0.929411768913269 0.013137091532158985 0.039411274596476954
test f1pos 0.9166810681526728 0.00443638831381126 0.013309164941433782
test f1neg 0.9108463086493312 0.0045307237111461484 0.01

In [254]:
eCNN_dict['set_3']

{'train': {'loss': [1.255494236946106,
   1.250248670578003,
   1.2720798254013062,
   1.3099104166030884,
   1.286524772644043,
   1.275626301765442,
   1.2598332166671753,
   1.2393217086791992,
   1.2509788274765015,
   1.2671191692352295],
  'auc': [0.9518687129020691,
   0.9480991959571838,
   0.9477150440216064,
   0.9497239589691162,
   0.9449139833450317,
   0.9472668170928955,
   0.9472509026527405,
   0.9458423256874084,
   0.9459784030914307,
   0.9457062482833862],
  'prec': [0.9189189076423645,
   0.8991596698760986,
   0.8804780840873718,
   0.8243727684020996,
   0.8695651888847351,
   0.8671875,
   0.9115044474601746,
   0.9099099040031433,
   0.831501841545105,
   0.8943089246749878],
  'acc': [0.8820000290870667,
   0.8899999856948853,
   0.8920000195503235,
   0.871999979019165,
   0.8840000033378601,
   0.8859999775886536,
   0.8820000290870667,
   0.8740000128746033,
   0.871999979019165,
   0.8980000019073486],
  'rec': [0.8326530456542969,
   0.8734694123268127,
